In [2]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime
from holidays_es import get_provinces, Province

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

2022-08-31 15:39:14.530463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-31 15:39:14.530517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Importing data

In [3]:
df = pd.read_json("../raw_data/202106.json", encoding_errors="ignore", lines=True)
weather = pd.read_csv("../raw_data/weather/weather_cleaned.csv")
df.head()

,_id,stations
0,2021-06-01T00:43:39.688214,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
1,2021-06-01T01:43:41.872092,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
2,2021-06-01T02:43:42.969250,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
3,2021-06-01T03:43:44.600717,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
4,2021-06-01T04:43:45.629938,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."


## Adding columns

In [4]:
df["time"] = "time"
df["date"] = "date"
for i in range(len(df['_id'])):
    item=df['_id'][i]
    groups= item.split("T")
    date = groups[0]
    raw_time = groups[1].split(":")
    time =':'.join(raw_time[:2])
    df["time"][i] = time
    df["date"][i] = date

In [5]:
len(df["stations"][1])

264

In [6]:
df['time'] = df.apply(lambda x:datetime.strptime(x['time'], '%H:%M').hour, axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [7]:
holidays=[]
for year in [2018,2019,2020,2021]:
    temp = Province(name="madrid", year=year).holidays()
    for day in range(len(temp['national_holidays'])):
        holidays.append(temp['national_holidays'][day])
    for day in range(len(temp['regional_holidays'])):
        holidays.append(temp['regional_holidays'][day])
    for day in range(len(temp['local_holidays'])):
        holidays.append(temp['local_holidays'][day])

In [8]:
df_holidays=pd.DataFrame(holidays, columns=['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
list_holidays=list(df_holidays['date'])
df['holidays']=df['date'].isin(list_holidays)

In [9]:
set(df.holidays)

{False}

# THIS TAKES A LOT TO RUN

In [10]:
len(df["stations"][172])

264

In [25]:
temp=pd.DataFrame()
for i in range(len(df)):
        new_dict= df['stations'][i][42]
        new_dict['id']=df['_id'][i]
        new_dict['time']=int(df['time'][i])
        new_dict['date']=df['date'][i]
        new_dict['holidays']=df['holidays'][i]
        temp = temp.append(new_dict, ignore_index=True)


In [26]:
temp["datetime"] = df["date"].astype(str)+df["time"].astype(str)
temp

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays,datetime
0,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,20.0,2021-06-01T00:43:39.688214,0.0,2021-06-01,0.0,2021-06-010
1,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T01:43:41.872092,1.0,2021-06-01,0.0,2021-06-011
2,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T02:43:42.969250,2.0,2021-06-01,0.0,2021-06-012
3,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T03:43:44.600717,3.0,2021-06-01,0.0,2021-06-013
4,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T04:43:45.629938,4.0,2021-06-01,0.0,2021-06-014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,1.0,Santa Isabel,1.0,0.0,24.0,18.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,5.0,2021-06-30T19:03:28.471971,19.0,2021-06-30,0.0,2021-06-3019
708,1.0,Santa Isabel,0.0,0.0,24.0,22.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,1.0,2021-06-30T20:03:28.506666,20.0,2021-06-30,0.0,2021-06-3020
709,1.0,Santa Isabel,0.0,2.0,24.0,13.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,10.0,2021-06-30T21:03:29.744034,21.0,2021-06-30,0.0,2021-06-3021
710,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,22.0,2021-06-30T22:03:31.966720,22.0,2021-06-30,0.0,2021-06-3022


In [27]:
weather

,feels_like,weather_main,datetime
0,3.08,Clear,2018-01-010
1,2.88,Clear,2018-01-011
2,2.11,Clear,2018-01-012
3,2.48,Clear,2018-01-013
4,1.69,Clear,2018-01-014
...,...,...,...
41488,28.51,Clear,2022-08-2719
41489,27.64,Clear,2022-08-2720
41490,27.38,Clear,2022-08-2721
41491,26.65,Clear,2022-08-2722


In [14]:
temp = temp.merge(weather, on="datetime")
temp

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays,datetime,feels_like,weather_main
0,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,20.0,2021-06-01T00:43:39.688214,0.0,2021-06-01,0.0,2021-06-010,17.44,Rain
1,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T01:43:41.872092,1.0,2021-06-01,0.0,2021-06-011,17.18,Clouds
2,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T02:43:42.969250,2.0,2021-06-01,0.0,2021-06-012,16.82,Thunderstorm
3,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T02:43:42.969250,2.0,2021-06-01,0.0,2021-06-012,16.82,Rain
4,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,21.0,2021-06-01T03:43:44.600717,3.0,2021-06-01,0.0,2021-06-013,16.76,Rain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,1.0,Santa Isabel,1.0,0.0,24.0,18.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,5.0,2021-06-30T19:03:28.471971,19.0,2021-06-30,0.0,2021-06-3019,28.84,Clear
709,1.0,Santa Isabel,0.0,0.0,24.0,22.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,1.0,2021-06-30T20:03:28.506666,20.0,2021-06-30,0.0,2021-06-3020,27.10,Clear
710,1.0,Santa Isabel,0.0,2.0,24.0,13.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,10.0,2021-06-30T21:03:29.744034,21.0,2021-06-30,0.0,2021-06-3021,26.05,Clear
711,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,40.4107085,22.0,2021-06-30T22:03:31.966720,22.0,2021-06-30,0.0,2021-06-3022,24.83,Clear


In [15]:

temp['weekday'] = temp.apply(lambda x: x['date'].weekday(), axis=1)
temp['year'] = temp.apply(lambda x:x['date'].year, axis=1)
temp['month'] = temp.apply(lambda x: x['date'].month, axis=1)

In [16]:
temp.head(2)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,id,time,date,holidays,datetime,feels_like,weather_main,weekday,year,month
0,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,...,2021-06-01T00:43:39.688214,0.0,2021-06-01,0.0,2021-06-010,17.44,Rain,1,2021,6
1,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,...,2021-06-01T01:43:41.872092,1.0,2021-06-01,0.0,2021-06-011,17.18,Clouds,1,2021,6


### Encoding circular time
temp['hour_sin'] = temp.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['season_sin'] = temp.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
temp['season_cos'] = temp.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [17]:
temp['hour_sin'] = temp.apply(lambda x: sin(x['time'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['time'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month']- 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5 ) % 12) / 12.0 * 2 * pi), axis=1)

In [18]:
temp.month

0      6
1      6
2      6
3      6
4      6
      ..
708    6
709    6
710    6
711    6
712    6
Name: month, Length: 713, dtype: int64

In [19]:
temp.drop(columns=["name","longitude","address","month","time","weekday"])
temp.head(3)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,weather_main,weekday,year,month,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos
0,1.0,Santa Isabel,0.0,1.0,24.0,1.0,42,-3.6982318,0.0,Calle Atocha nº 95,...,Rain,1,2021,6,0.000000,1.000000,0.781831,0.62349,0.5,0.866025
1,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,...,Clouds,1,2021,6,0.258819,0.965926,0.781831,0.62349,0.5,0.866025
2,1.0,Santa Isabel,0.0,1.0,24.0,0.0,42,-3.6982318,0.0,Calle Atocha nº 95,...,Thunderstorm,1,2021,6,0.500000,0.866025,0.781831,0.62349,0.5,0.866025


In [20]:
#temp.to_csv("filename.processed.csv")